In [1]:
from pyspark.sql import SparkSession, types as T, functions as F
from pyspark import SparkConf

conf = SparkConf().setAppName("learn")
conf.set('spark.jars.packages', 'io.delta:delta-core_2.12:2.1.0')
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1ee94148-0420-4c53-bf6a-624101712f51;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 885ms :: artifacts dl 45ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnl

In [2]:
schema_redemer = T.StructType([T.StructField(nm, T.StringType(), True) for nm in ['Sr_No', 'Date_of_Encashment', 'Name_of_the_Political_Party', 'Account_no._of_Political_Party', 'Prefix', 'Bond_Number', 'redemer_Denominations', 'Pay_Branch_Code', 'Pay_Teller'] ])
schema_purchaser = T.StructType([T.StructField(nm, T.StringType(), True) for nm in ['Sr_No', 'Reference_No_(URN)', 'Journal_Date', 'Date_of_Purchase', 'Date_of_Expiry', 'Name_of_the_Purchaser', 'Prefix', 'Bond_Number', 'purchaser_Denominations', 'Issue_Branch_Code', 'Issue_Teller', 'Status'] ])
df_redemer = spark.read.format('csv').schema(schema_redemer).load('/home/glue_user/workspace/data-engineering/data/source/src-data-csv/electoral-bonds/bond_redemer_details.csv').filter("Pay_Teller is not null")
df_purchaser = spark.read.format('csv').schema(schema_purchaser).load('/home/glue_user/workspace/data-engineering/data/source/src-data-csv/electoral-bonds/bond_purchaser_details.csv').filter("purchaser_Denominations is not null")

df_redemer.createOrReplaceTempView("redemer")
df_purchaser.createOrReplaceTempView("purchaser")

df_redemer.printSchema()
df_purchaser.printSchema()

root
 |-- Sr_No: string (nullable = true)
 |-- Date_of_Encashment: string (nullable = true)
 |-- Name_of_the_Political_Party: string (nullable = true)
 |-- Account_no._of_Political_Party: string (nullable = true)
 |-- Prefix: string (nullable = true)
 |-- Bond_Number: string (nullable = true)
 |-- redemer_Denominations: string (nullable = true)
 |-- Pay_Branch_Code: string (nullable = true)
 |-- Pay_Teller: string (nullable = true)

root
 |-- Sr_No: string (nullable = true)
 |-- Reference_No_(URN): string (nullable = true)
 |-- Journal_Date: string (nullable = true)
 |-- Date_of_Purchase: string (nullable = true)
 |-- Date_of_Expiry: string (nullable = true)
 |-- Name_of_the_Purchaser: string (nullable = true)
 |-- Prefix: string (nullable = true)
 |-- Bond_Number: string (nullable = true)
 |-- purchaser_Denominations: string (nullable = true)
 |-- Issue_Branch_Code: string (nullable = true)
 |-- Issue_Teller: string (nullable = true)
 |-- Status: string (nullable = true)



In [ ]:
df_redemer.select(F.expr("sum(cast(replace(redemer_Denominations, ',', '') as bigint)) over(partition by Name_of_the_Political_Party)").alias('party_wise_donation'),
                    F.expr("Name_of_the_Political_Party")).show()

In [ ]:
def str_to_inr(s: str) -> str :
    n = int(s)
    q = n%1000
    s1 = str(n%1000)
    n = int(n/1000)
    s2 = ''
    while(n):
        r = n%100
        n = int(n/100)
        s2 = str(r) + ',' + s2
    return s2+s1
str_to_inr = F.udf(str_to_inr, T.StringType())
spark.udf.register("str_to_inr", str_to_inr)

In [ ]:
df_redemer.withColumn('redemer_Denominations', 
                      F.expr("cast(replace(redemer_Denominations, ',', '') as bigint)")).orderBy(F.expr('redemer_Denominations').desc()).show()

In [ ]:
# Define the schema for each table
users_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("name", T.StringType(), True)
])

addresses_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("street", T.StringType(), True),
    T.StructField("city", T.StringType(), True),
    T.StructField("postal_code", T.StringType(), True)
])

favorite_products_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("product_id", T.StringType(), True)
])

preferences_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("key", T.StringType(), True),
    T.StructField("value", T.StringType(), True)
])

extra_info_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("field1", T.StringType(), True),
    T.StructField("field2", T.StringType(), True)
])

additional_info_schema = T.StructType([
    T.StructField("user_id", T.StringType(), True),
    T.StructField("field3", T.StringType(), True),
    T.StructField("field4", T.StringType(), True)
])

# Define data for each table
users_data = [
    ("U123", "John Doe"),
    ("U124", "Jane Doe"),
    ("U125", "Alice Smith"),
    ("U126", "Bob Brown")
]

addresses_data = [
    ("U123", "123 Elm St", "Somewhere", "12345"),
    ("U123", "456 Oak St", "Anywhere", "67890"),
    ("U124", "789 Pine St", "Somewhere", "12345"),
    ("U125", "101 Maple St", "Elsewhere", "54321"),
    ("U126", "202 Birch St", "Nowhere", "98765")
]

favorite_products_data = [
    ("U123", "P001"),
    ("U123", "P002"),
    ("U123", "P003"),
    ("U124", "P004"),
    ("U124", "P005"),
    ("U125", "P006"),
    ("U126", "P007"),
    ("U126", "P008"),
    ("U126", "P009"),
    ("U126", "P010")
]

preferences_data = [
    ("U123", "newsletter", "subscribed"),
    ("U123", "theme", "dark"),
    ("U124", "newsletter", "unsubscribed"),
    ("U124", "theme", "light"),
    ("U124", "language", "en"),
    ("U125", "theme", "dark"),
    ("U126", "language", "fr"),
    ("U126", "currency", "euro"),
    ("U126", "newsletter", "subscribed")
]

extra_info_data = [
    ("U123", "foo", "bar"),
    ("U124", "baz", "qux"),
    ("U125", "abc", "def"),
    ("U126", "ghi", "jkl")
]

additional_info_data = [
    ("U123", "foo1", "bar1"),
    ("U124", "baz1", "qux1"),
    ("U125", "abc1", "def1"),
    ("U126", "ghi1", "jkl1")
]

# Create DataFrames for each table
users_df = spark.createDataFrame(users_data, schema=users_schema)
addresses_df = spark.createDataFrame(addresses_data, schema=addresses_schema)
favorite_products_df = spark.createDataFrame(favorite_products_data, schema=favorite_products_schema)
preferences_df = spark.createDataFrame(preferences_data, schema=preferences_schema)
extra_info_df = spark.createDataFrame(extra_info_data, schema=extra_info_schema)
additional_info_df = spark.createDataFrame(additional_info_data, schema=additional_info_schema)

# Show DataFrames
print("users:")
users_df.show()

print("addresses:")
addresses_df.show()

print("favorite_products:")
favorite_products_df.show()

print("favorite_products:")
preferences_df.show()

print("extra_info:")
extra_info_df.show()

print("additional_info:")
additional_info_df.show()

users_df.createOrReplaceTempView("users")
addresses_df.createOrReplaceTempView("addresses")
favorite_products_df.createOrReplaceTempView("favorite_products")
preferences_df.createOrReplaceTempView("preferences")
extra_info_df.createOrReplaceTempView("extra_info")
additional_info_df.createOrReplaceTempView("additional_info")

In [ ]:
sqlQuery = '''
select
    users.user_id,
    array(
        street,
        city,
        postal_code
    ) address
from
    users
    left join addresses on users.user_id=addresses.user_id
    left join favorite_products on favorite_products.user_id=addresses.user_id
'''
df_output = spark.sql(sqlQuery)
df_output.show(truncate=False)
df_output.printSchema()
df_output.toJSON().collect()
df_output.createOrReplaceTempView('op')

In [ ]:
spark.sql('''
select *, explode(address) from op          
''').show()

In [ ]:
sqlQuery = '''
SELECT
    c2,
    collect_list(c1)
FROM (
    VALUES 
        (1, 'A'),
        (1, 'A'),
        (1, 'A'),
        (2, 'B'),
        (2, 'B'), 
        (3, 'C')
) AS tab(c1, c2)
group by c2
'''
spark.sql(sqlQuery).printSchema()

In [ ]:
sqlQuery = '''
SELECT
    map(c1, c2) cl
FROM (
    VALUES 
        (1, 'A'),
        (1, 'A'),
        (1, 'A'),
        (2, 'B'),
        (2, 'B'), 
        (3, 'C')
) AS tab(c1, c2)
'''
spark.sql(sqlQuery).show()

In [ ]:
# Create sample DataFrames
df1 = spark.createDataFrame([("A", 1), ("B", 2), ("C", 3)], ["id", "value1"])
df2 = spark.createDataFrame([("A", 4), ("B", 5), ("D", 6)], ["id", "value2"])

# Join using expr for complex condition
df1.join()

In [ ]:
# Sample data with nested structure
data = [(1, [1, 2, 3]), (2, [4, 5])]

# Create DataFrame
df = spark.createDataFrame(data, ["id", "values"])

# Explode the array column
df.show()
df_flattened = df.select("id", F.explode("values").alias("value"))

# Show the flattened DataFrame
df_flattened.show()

In [ ]:
#corrupted records
df = spark.read.format('csv').options(schema=['id', 'name', '_corrupt_records'], header='true', mode='PERMISSIVE', badRecordsPath='/home/glue_user/workspace/data-engineering/tmp/badrecords').load('C:\Users\raulr\OneDrive\myProjects\dockerBind\awsglue\data-engineering\data\source\src-data-csv-corrupted\actor.csv')

In [ ]:
df.show()

In [ ]:
class Animal():
    def __init__(self, hands, legs):
        self.hands = hands
        self.legs = legs
    def run(self, speed):
        return f"running at speed {speed}"

In [ ]:
human = Animal(2, 2)
human.run(10)

In [ ]:
df_redemer.printSchema()

In [ ]:
df_redemer.createOrReplaceTempView('redemer')
spark.sql('''
select distinct redemer_Denominations from redemer
''').show()

In [ ]:
schema = T.StructType([
    T.StructField('name', T.StringType(), True),
    T.StructField('age', T.IntegerType(), False)
])

dir(schema)

In [ ]:

# df = spark.read.format('csv').options(schema=['id', 'name', '_corrupt_records'], header='true', mode='PERMISSIVE', badRecordsPath='/home/glue_user/workspace/data-engineering/tmp/badrecords').load('C:\Users\raulr\OneDrive\myProjects\dockerBind\awsglue\data-engineering\data\source\src-data-csv-corrupted\actor.csv')
schema = 
df = spark.read.schema('id int,name string, _corrupt_record string').format('csv').options(columnNameOfCorruptRecord = "_corrupt_record", header='true', mode = "PERMISSIVE").load('/home/glue_user/workspace/data-engineering/data/source/src-data-csv-corrupted/actor.csv')
df.show()

In [ ]:
df.filter(F.expr("_corrupt_record is not Null")).show()

In [ ]:
def f(a, b):
    return a+b

a = [1, 2]

m = map(f, a, a)

list(m)

In [ ]:
def ff(a):
    if a in [1, 3, 5, 7]:
        return True
    else:
        return False
b = [i for i in range(0, 10)]
list(filter(ff, b))

In [ ]:
from pyspark.sql import SparkSession, types as T, functions as F
from pyspark import SparkConf

conf = SparkConf()

conf.setAppName('prac')

conf = SparkConf().setAppName("learn")
conf.set('spark.jars.packages', 'io.delta:delta-core_2.12:2.1.0')
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
data = ['a', 'b', 1, 2, ['x', 'y', 3]]
data = [i for i in range(0, 100)]
data = ["hello world", "spark is awesome", []]
#data = [('k', 1), ('b', 1), ('c', 1), ('a', 1), ('b', 1), ('c', 1)]
rdd = spark.sparkContext.parallelize(data)
rdd.collect()

In [ ]:
#rdd.reduceByKey(lambda x, y: x+y).collect()
rdd.sortByKey().collect()

In [80]:
def pyfun(i):
    import uuid
    return str(uuid.uuid4())

dapi_udf = F.udf(pyfun, T.StringType())
spark.udf.register("sql_func", pyfun, T.StringType())

24/08/26 17:30:08 WARN SimpleFunctionRegistry: The function suuid_udf replaced a previously registered function.


<function __main__.suuid(i)>

In [81]:
df = spark.range(100).withColumn('dec', F.col('id').cast(T.DecimalType(10, 2)))\
    .withColumn('uuids', suuid_udf(F.col('id')))
df.show()

+---+-----+--------------------+
| id|  dec|               uuids|
+---+-----+--------------------+
|  0| 0.00|740b074c-6560-4e9...|
|  1| 1.00|0bb1016c-a6e3-41e...|
|  2| 2.00|13b40418-3d69-43a...|
|  3| 3.00|445ecf6a-a3d3-43f...|
|  4| 4.00|4ae654f3-206f-433...|
|  5| 5.00|09ea3428-ae8d-4bd...|
|  6| 6.00|d6dcbc7f-d8a2-415...|
|  7| 7.00|7848cc79-1286-459...|
|  8| 8.00|a82a1ccf-4ecc-4e7...|
|  9| 9.00|01cb6fa9-0dad-4a5...|
| 10|10.00|ca70f54f-aafd-409...|
| 11|11.00|c228625a-6f0b-4e0...|
| 12|12.00|7eb88830-b026-4f6...|
| 13|13.00|cf9b3b17-1f4b-42c...|
| 14|14.00|a12d220e-7cc6-481...|
| 15|15.00|255fc2bf-1667-44b...|
| 16|16.00|a766e1dc-101f-4e7...|
| 17|17.00|11043d2e-df85-46d...|
| 18|18.00|e9e34194-df90-418...|
| 19|19.00|efdbd34c-4d56-46e...|
+---+-----+--------------------+
only showing top 20 rows



In [97]:
df.repartition(1).write.format('parquet').bucketBy(10, "id").saveAsTable("bucketed_table4")

24/08/26 18:05:20 INFO FileUtils: Creating directory if it doesn't exist: file:/home/glue_user/workspace/data-engineering/practice/spark-warehouse/bucketed_table4
24/08/26 18:05:20 INFO HdfsUtils: Skipping ACL inheritance: File system for path file:/home/glue_user/workspace/data-engineering/practice/spark-warehouse does not support ACLs but dfs.namenode.acls.enabled is set to true. 


In [92]:
dff = spark.sql('''
select * from bucketed_table
          ''')
dff.rdd.getNumPartitions()

4

In [118]:
spark.sql('''
select * from (select *, () over(order by prefix) rn from redemer) order by rn
''').show(1000)

24/08/27 05:08:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:08:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:08:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:08:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:08:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:08:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 0

+-----+------------------+---------------------------+------------------------------+------+-----------+---------------------+---------------+----------+---+
|Sr_No|Date_of_Encashment|Name_of_the_Political_Party|Account_no._of_Political_Party|Prefix|Bond_Number|redemer_Denominations|Pay_Branch_Code|Pay_Teller| rn|
+-----+------------------+---------------------------+------------------------------+------+-----------+---------------------+---------------+----------+---+
|    1|       12/Apr/2019|       ALL INDIA ANNA DR...|                   *******5199|    OC|        775|          1,00,00,000|          00800|   2770121|  1|
|    2|       12/Apr/2019|       ALL INDIA ANNA DR...|                   *******5199|    OC|       3975|          1,00,00,000|          00800|   2770121|  1|
|    3|       12/Apr/2019|       ALL INDIA ANNA DR...|                   *******5199|    OC|       3967|          1,00,00,000|          00800|   2770121|  1|
|   34|       12/Apr/2019|       BHARAT RASHTRA SA..

In [116]:
data = [
    ("Alice", 5000),
    ("Bob", 4000),
    ("Charlie", 4000),
    ("David", 3000),
    ("Eve", 2500)
]

# Create DataFrame
df = spark.createDataFrame(data, ["name", "salary"])

# Create a temporary view
df.createOrReplaceTempView("employee")

# Run SQL query with RANK() and DENSE_RANK()
result = spark.sql("""
    SELECT 
        name, 
        salary, 
        RANK() OVER (ORDER BY salary DESC) AS rank, 
        DENSE_RANK() OVER (ORDER BY salary DESC) AS dense_rank 
    FROM 
        employee
""")

result.show()

24/08/27 05:05:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:05:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:05:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:05:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/27 05:05:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+------+----+----------+
|   name|salary|rank|dense_rank|
+-------+------+----+----------+
|  Alice|  5000|   1|         1|
|    Bob|  4000|   2|         2|
|Charlie|  4000|   2|         2|
|  David|  3000|   4|         3|
|    Eve|  2500|   5|         4|
+-------+------+----+----------+



In [126]:
spark.sql('''
select to_date('2024/12/01', 'yyyy/MM/dd') td
''').show()

+----------+
|        td|
+----------+
|2024-12-01|
+----------+



In [137]:
spark.sql('''
select date_format(current_timestamp(), 'HH:mm:ss.SSS a x') sd
''').show(truncate=False)

+-------------------+
|sd                 |
+-------------------+
|05:25:19.601 AM +00|
+-------------------+



In [138]:
data = [("John", "Order1"), ("John", "Order2"), ("Jane", "Order3"), ("Jane", "Order4"), ("Jake", "Order5")]
df = spark.createDataFrame(data, ["Customer", "Order"])

In [141]:
df.withColumn('cnt', F.expr('count(*) over(partition by customer)')).show()

+--------+------+---+
|Customer| Order|cnt|
+--------+------+---+
|    Jake|Order5|  1|
|    Jane|Order3|  2|
|    Jane|Order4|  2|
|    John|Order1|  2|
|    John|Order2|  2|
+--------+------+---+



In [7]:
df_redemer.repartition(1).write.mode('overwrite').options(header='true').format('csv').save('/home/glue_user/workspace/data-engineering/data/target/output/redemer')

In [8]:
df_redemer.show()

+-----+------------------+---------------------------+------------------------------+------+-----------+---------------------+---------------+----------+
|Sr_No|Date_of_Encashment|Name_of_the_Political_Party|Account_no._of_Political_Party|Prefix|Bond_Number|redemer_Denominations|Pay_Branch_Code|Pay_Teller|
+-----+------------------+---------------------------+------------------------------+------+-----------+---------------------+---------------+----------+
|    1|       12/Apr/2019|       ALL INDIA ANNA DR...|                   *******5199|    OC|        775|          1,00,00,000|          00800|   2770121|
|    2|       12/Apr/2019|       ALL INDIA ANNA DR...|                   *******5199|    OC|       3975|          1,00,00,000|          00800|   2770121|
|    3|       12/Apr/2019|       ALL INDIA ANNA DR...|                   *******5199|    OC|       3967|          1,00,00,000|          00800|   2770121|
|    4|       12/Apr/2019|       ALL INDIA ANNA DR...|                   ***